In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

In [ ]:
WORKING_DIR = os.getcwd()  # Define main working directory for safety


def is_safe_path(path: str) -> bool:
    """Ensure the path stays within the main working directory."""
    abs_path = os.path.abspath(path)
    return abs_path.startswith(WORKING_DIR) and ".." not in path


@tool
def create_directory(dirname: str):
    """Creates a directory with the given name (safely within working dir)."""
    if not is_safe_path(dirname):
        return f"Unsafe path detected: '{dirname}'"

    try:
        os.makedirs(dirname, exist_ok=True)
        return f"Directory '{dirname}' created successfully (or already exists)."
    except Exception as e:
        return f"Error creating directory '{dirname}': {e}"


@tool
def create_or_edit_file(filepath: str, content: str = ""):
    """Creates or overwrites a file with optional content."""
    if not is_safe_path(filepath):
        return f"Unsafe path detected: '{filepath}'"

    try:
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"File '{filepath}' created successfully."
    except Exception as e:
        return f"Error creating file '{filepath}': {e}"


@tool
def read_file(filepath: str):
    """Reads and returns the content of a file (safely)."""
    if not is_safe_path(filepath):
        return f"Unsafe path detected: '{filepath}'"

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file '{filepath}': {e}"


@tool
def edit_file_lines(filepath: str, start_line: int, end_line: int, new_content: str):
    """
    Replaces a range of lines in a file with new content.
    
    Parameters:
    - filepath: Path to the target file.
    - start_line: 1-based index of the first line to replace.
    - end_line: 1-based index of the last line to replace (inclusive).
    - new_content: Replacement text (can be multiple lines).
    
    Behavior:
    - Replaces all lines in the specified range with `new_content`.
    - If new_content is empty, deletes the lines.
    - Safe: will not modify files outside the working directory.
    - Prints a log message for transparency.
    """
    if not is_safe_path(filepath):
        print(f"[LOG] Unsafe path attempt blocked: {filepath}")
        return f"Unsafe path detected: '{filepath}'"

    try:
        print(f"[LOG] Editing lines {start_line}-{end_line} in '{filepath}'")

        # Read file
        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()

        if start_line < 1 or end_line < start_line or end_line > len(lines):
            print(f"[LOG] Invalid range {start_line}-{end_line}. File has {len(lines)} lines.")
            return f"Invalid range {start_line}-{end_line}. File has {len(lines)} lines."

        # Split new content into normalized lines
        new_lines = []
        if new_content.strip() != "":
            new_lines = [l if l.endswith("\n") else l + "\n" for l in new_content.splitlines()]

        # Replace range
        lines = lines[: start_line - 1] + new_lines + lines[end_line:]

        # Write updated file
        with open(filepath, "w", encoding="utf-8") as f:
            f.writelines(lines)

        action = "replaced" if new_lines else "deleted"
        print(f"[LOG] Lines {start_line}-{end_line} {action} successfully in '{filepath}'.")
        return f"Lines {start_line}-{end_line} in '{filepath}' {action} successfully."

    except Exception as e:
        print(f"[LOG] Error editing lines {start_line}-{end_line} in '{filepath}': {e}")
        return f"Error editing lines {start_line}-{end_line} in '{filepath}': {e}"


tools = [create_directory, create_or_edit_file, read_file, edit_file_lines]


In [ ]:
llm = ChatGroq(model="openai/gpt-oss-120B", temperature=0.2).bind_tools(tools)

system_prompt = SystemMessage(
    content=(
        """
        You are a **File System Assistant**.
        Your job is to manage files and directories using the provided tools.
        
        **Responsibilities:**
        - Create, read, and edit files or folders as instructed by the user.
        - Use the available tools only — do not execute code or access the OS directly.
        - Make precise, minimal edits when requested (e.g., update specific lines).

        **Safety Rules:**
        - Never access or modify files outside the main working directory.
        - Never use relative paths containing '..' or absolute paths outside the workspace.
        - Handle all errors gracefully and report clear messages back to the user.

        **Goal:**
        - Efficiently perform file operations within the sandboxed environment,
          ensuring safety, accuracy, and reliability.
        """
    )
)

In [ ]:
def llm_node(state: MessagesState):
    """LLM node: generates next message or calls tool."""
    response = llm.invoke([system_prompt] + state["messages"])
    return {"messages": [system_prompt] + state["messages"] + [response]}


tool_node = ToolNode(tools)

In [ ]:
graph = StateGraph(MessagesState)
graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)

graph.add_conditional_edges(
    "llm",
    lambda state: (
        "tools" if state["messages"][-1].tool_calls else END
    ),
)

graph.add_edge("tools", "llm")
graph.add_edge(START, "llm")

assistant_graph = graph.compile()

In [20]:
global_messages = []
while True:
    question = input("What do you want sir? : ")
    print("Query: " + question)
    global_messages.append(HumanMessage(content=question))
    result = assistant_graph.invoke(MessagesState(messages=global_messages))
    global_messages = result["messages"]
    final_message = result["messages"][-1].content
    print("\nFinal Answer:", final_message)

Query: add 2 more dummy endpoints
[LOG] Editing lines 20-24 in 'demo/src/main/java/com/example/demo/DemoController.java'
[LOG] Lines 20-24 replaced successfully in 'demo/src/main/java/com/example/demo/DemoController.java'.

Final Answer: The controller now includes two additional dummy endpoints.

**Updated `DemoController.java`:**

```java
package com.example.demo;

import org.springframework.web.bind.annotation.*;
import org.springframework.http.ResponseEntity;

@RestController
@RequestMapping("/api")
public class DemoController {

    @GetMapping("/hello")
    public ResponseEntity<String> hello() {
        return ResponseEntity.ok("Hello, World!");
    }

    @PostMapping("/echo-here")
    public ResponseEntity<String> echo(@RequestBody String body) {
        return ResponseEntity.ok(body);
    }

    @DeleteMapping("/remove/{id}")
    public ResponseEntity<String> remove(@PathVariable String id) {
        // dummy implementation
        return ResponseEntity.ok("Removed item with 

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `openai/gpt-oss-120b` in organization `org_01k8tc7jt0e4srpfn7b2zfc3vk` service tier `on_demand` on tokens per minute (TPM): Limit 8000, Requested 8056, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}